In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_dir = 'omr.v6i.folder/train/'
test_dir = 'omr.v6i.folder/test/'
val_dir = 'omr.v6i.folder/valid/'

In [ ]:
data_train = image_dataset_from_directory(train_dir,labels='inferred',
                                       label_mode='categorical',interpolation='nearest',image_size=[150,150],batch_size=32,
                                       shuffle=True)
data_validation = image_dataset_from_directory(val_dir,labels='inferred',
                                       label_mode='categorical',interpolation='nearest',image_size=[150,150],batch_size=32,
                                       shuffle=True)
data_test = image_dataset_from_directory(test_dir,labels='inferred',
                                       label_mode='categorical',interpolation='nearest',image_size=[150,150],batch_size=32,
                                       shuffle=True)

In [ ]:
classes_train = data_train.class_names
classes_train

In [ ]:
IMG_SIZE = (256, 256)

# data argumentation
train_datagen = ImageDataGenerator(rescale=1/255.0,
                                       rotation_range=45,
                                       shear_range=0.2,
                                       zoom_range=0.2,
                                       horizontal_flip=True,
                                       vertical_flip=True
                                       )

# data argumentation
test_datagen = ImageDataGenerator(rescale= 1 / 255.0)

train_dataset = train_datagen.flow_from_directory(train_dir, target_size=(IMG_SIZE), 
                                         color_mode="rgb",
                                         batch_size=16, 
                                         shuffle=True,
                                         class_mode="categorical")

test_dataset = test_datagen.flow_from_directory(test_dir, target_size=(IMG_SIZE), 
                                         color_mode="rgb",
                                         batch_size=16, 
                                         shuffle=True,
                                         class_mode="categorical")

validation_dataset = train_datagen.flow_from_directory(val_dir, target_size=(IMG_SIZE), 
                                         color_mode="rgb",
                                         batch_size=16, 
                                         shuffle=True,
                                         class_mode="categorical")

In [ ]:
# using resnet152v2 model
resnet152v2 = ResNet152V2(weights='imagenet',include_top=False,input_shape=(256,256,3))
# make pre trained model into non trainable bcoz its takes much time
for layer in resnet152v2.layers:
    layer.trainable = False

In [ ]:
# Initialize the model in sequential
model = Sequential()
# add resnet101v2 model into our sequence model
model.add(resnet152v2)
# flatten the model
model.add(Flatten())
# Adding dense layers
model.add(Dense(128, activation="relu", kernel_initializer="he_uniform"))
# Adding output layer
model.add(Dense(5,activation='sigmoid'))
model.summary()

In [ ]:
# model compile
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
history = model.fit(train_dataset,
                    validation_data=validation_dataset,
                    epochs=1,
                    verbose=1)

In [ ]:
def img_preprocessing(path):
    sample_mask_img = cv2.imread(path)
    sample_mask_img = cv2.resize(sample_mask_img,(256,256))
    plt.imshow(sample_mask_img)
    sample_mask_img = np.reshape(sample_mask_img,[1,256,256,3])
    sample_mask_img = sample_mask_img/255.0
    return sample_mask_img

In [ ]:
img1 = img_preprocessing('omr.v6i.folder/test/4/th5115-jpg_8_jpg.rf.a6624367b8b247e200d663823f4b4b66.jpg')
np.argmax(model.predict(img1))